 ### 01 Import Libraries

In [2]:
import pandas as pd
import numpy as np
import os

###  02 Import Data

In [3]:
# import ords_prods_merge
df_opm = pd.read_pickle(r"C:\Users\drmat\OneDrive\Documents\Instacart Basket Analysis\02 Data\Prepared Data\orders_products_merged4-7.pkl")

In [4]:
# check data
df_opm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 20 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   Unnamed: 0_x            int64   
 1   order_id                int64   
 2   user_id                 int64   
 3   order_number            int64   
 4   order_day_or_week       int64   
 5   order_hour_of_day       int64   
 6   days_since_prior_order  float64 
 7   product_id              int64   
 8   add_to_cart_order       int64   
 9   reordered               int64   
 10  first_merge             category
 11  Unnamed: 0_y            int64   
 12  product_name            object  
 13  aisle_id                int64   
 14  department_id           int64   
 15  prices                  float64 
 16  _merge                  category
 17  price_range             object  
 18  busiest_day             object  
 19  busiest_period_of_day   object  
dtypes: category(2), float64(2), int64(12), objec

###  03 Question 2

In [5]:
# aggregated mean of the “order_number” column grouped by “department_id"
df_opm.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### 03 Compare to df subset of 1000000

In [6]:
# create subset
df=df_opm[:1000000]

In [7]:
# aggregated mean of the “order_number” column grouped by “department_id"
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


How do the results for the entire dataframe differ from those of the subset?
The mean values of the entire dataframe are different from the department id's represented in the subset

###  04 Question 4

In [8]:
# create “max_order” column
df_opm['max_order'] = df_opm.groupby(['user_id'])['order_number'].transform(np.max)

In [9]:
# check data
df_opm.head()

,Unnamed: 0_x,order_id,user_id,order_number,order_day_or_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,Unnamed: 0_y,product_name,aisle_id,department_id,prices,_merge,price_range,busiest_day,busiest_period_of_day,max_order
0,0,2539329,1,1,2,8,NaN,196,1,0,...,195,Soda,77,7,9.0,both,mid-range price,regular busy day,Average orders,10
1,1,2398795,1,2,3,7,15.0,196,1,1,...,195,Soda,77,7,9.0,both,mid-range price,least busy day,Average orders,10
2,2,473747,1,3,3,12,21.0,196,1,1,...,195,Soda,77,7,9.0,both,mid-range price,least busy day,Most orders,10
3,3,2254736,1,4,4,7,29.0,196,1,1,...,195,Soda,77,7,9.0,both,mid-range price,least busy day,Average orders,10
4,4,431534,1,5,4,15,28.0,196,1,1,...,195,Soda,77,7,9.0,both,mid-range price,least busy day,Most orders,10


In [10]:
# create a flag that assigns a “loyalty” label to a user ID based on its corresponding max order value

In [10]:
df_opm.loc[df_opm['max_order']>40, 'loyalty_label']='Loyal Customer'

In [11]:
df_opm.loc[(df_opm['max_order']<=40)&(df_opm['max_order']>10), 'loyalty_label']='Regular Customer'

In [12]:
df_opm.loc[df_opm['max_order']<=10, 'loyalty_label']='New Customer'

In [13]:
# check data frequency
df_opm['loyalty_label'].value_counts(dropna=False)

Regular Customer    15876776
Loyal Customer      10284093
New Customer         6243990
Name: loyalty_label, dtype: int64

### 05 Question 5 Are the prices of products purchased by loyal customers differ from those purchased by regular or new customers?

In [15]:
# check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer)

In [14]:
df_opm.groupby('loyalty_label').agg({'prices':['mean', 'median']})

prices       
                       mean median
loyalty_label                     
Loyal Customer    10.386336    7.4
New Customer      13.294670    7.4
Regular Customer  12.495717    7.4

There is a small difference between average prices

###  06 Question 6 

In [15]:
# create “average_order” column
df_opm['average_order'] = df_opm.groupby(['user_id'])['prices'].transform(np.mean)

In [16]:
# check data
df_opm.head()

,Unnamed: 0_x,order_id,user_id,order_number,order_day_or_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,price_range,busiest_day,busiest_period_of_day,max_order,loyalty_label,average_order
0,0,2539329,1,1,2,8,NaN,196,1,0,...,77,7,9.0,both,mid-range price,regular busy day,Average orders,10,New Customer,6.367797
1,1,2398795,1,2,3,7,15.0,196,1,1,...,77,7,9.0,both,mid-range price,least busy day,Average orders,10,New Customer,6.367797
2,2,473747,1,3,3,12,21.0,196,1,1,...,77,7,9.0,both,mid-range price,least busy day,Most orders,10,New Customer,6.367797
3,3,2254736,1,4,4,7,29.0,196,1,1,...,77,7,9.0,both,mid-range price,least busy day,Average orders,10,New Customer,6.367797
4,4,431534,1,5,4,15,28.0,196,1,1,...,77,7,9.0,both,mid-range price,least busy day,Most orders,10,New Customer,6.367797


In [19]:
# Create a spending flag for each user based on the average price across all orders

In [17]:
df_opm.loc[df_opm['average_order']<10, 'spending_label']='Low Spender'

In [18]:
df_opm.loc[df_opm['average_order']>=10, 'spending_label']='High Spender'

In [19]:
# check data frequency
df_opm['spending_label'].value_counts(dropna=False)

Low Spender     31770614
High Spender      634245
Name: spending_label, dtype: int64

###  07 Question 7 Frequent versus non-frequent customer

In [20]:
# create median “ordering_behavior” column
df_opm['ordering_behavior'] = df_opm.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [21]:
# check data
df_opm.head()

,Unnamed: 0_x,order_id,user_id,order_number,order_day_or_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,_merge,price_range,busiest_day,busiest_period_of_day,max_order,loyalty_label,average_order,spending_label,ordering_behavior
0,0,2539329,1,1,2,8,NaN,196,1,0,...,9.0,both,mid-range price,regular busy day,Average orders,10,New Customer,6.367797,Low Spender,20.5
1,1,2398795,1,2,3,7,15.0,196,1,1,...,9.0,both,mid-range price,least busy day,Average orders,10,New Customer,6.367797,Low Spender,20.5
2,2,473747,1,3,3,12,21.0,196,1,1,...,9.0,both,mid-range price,least busy day,Most orders,10,New Customer,6.367797,Low Spender,20.5
3,3,2254736,1,4,4,7,29.0,196,1,1,...,9.0,both,mid-range price,least busy day,Average orders,10,New Customer,6.367797,Low Spender,20.5
4,4,431534,1,5,4,15,28.0,196,1,1,...,9.0,both,mid-range price,least busy day,Most orders,10,New Customer,6.367797,Low Spender,20.5


In [22]:
df_opm.loc[df_opm['ordering_behavior']>20, 'behavior_label']='Non-frequent customer'

In [23]:
df_opm.loc[df_opm['ordering_behavior']<=10, 'behavior_label']='Frequent customer'

In [24]:
df_opm.loc[(df_opm['ordering_behavior']>10)&(df_opm['ordering_behavior']<=20), 'behavior_label']='Regular customer'

In [25]:
# check data frequency
df_opm['behavior_label'].value_counts(dropna=False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: behavior_label, dtype: int64

### 08 Export Data

In [26]:
df_opm.to_pickle(r"C:\Users\drmat\OneDrive\Documents\Instacart Basket Analysis\02 Data\Prepared Data\orders_products_merged4-8.pkl")